# Deep Fool

Experiments with the DeepFool algorithm from [Moosavi-Dezfooli, Frossard](https://github.com/LTS4/DeepFool/tree/master/Python). In order to use this notebook the DeepFool implementation has to be downloaded and the Python folder saved to `modules`.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torchvision import models
from torchvision import transforms

from IPython.display import clear_output

# Import helper functions
from modules.helper import *
from modules.dataset import ImageNetSubset

# Import DeepFool
from modules.DeepFool import deepfool

## 0. Load Model and Data

We load the pre-trained googleNet and the data.

In [ ]:
# Get googleNet without the softmax layer

# Load pretrained net
model = models.googlenet(pretrained=True)

# Remove output layer
modules = list(model.children())[:-1]
googleNetMod = nn.Sequential(*modules)

for p in googleNetMod.parameters():
    p.requires_grad = False
    
googleNetMod.eval();

## Load full net for predictions later
googleNet = models.googlenet(pretrained=True)
googleNet.eval();

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]   

preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

data_loader = torch.utils.data.DataLoader(
    ImageNetSubset("data/ImageNet_subset//dev_dataset.csv", "data/ImageNet_subset//images/", transform=preprocess))

In [ ]:
###### BASIC STEPS FOR DEEPFOOL

# Parameters
sample = 766


# Get image
image_clean, class_index = data_loader.dataset[sample]
image_clean.unsqueeze_(0)
class_index.unsqueeze_(0)

# Change dimension from (1, 3, 224, 224) to (3, 224, 224):
image_clean.squeeze_(0)

## Deep Fool attack
r_tot, loop_i, class_name_clean, class_name_adver, pert_image = deepfool.deepfool(image_clean, googleNet)
print("Number of iterations: {}".format(loop_i))
print("Adversarial class name: {}".format(class_name_adver))


## Test the adversary:
pred_class, confidences, _ = predict(googleNet, pert_image, class_index)
print("\n\nTrue class index: " +str(int(class_index.numpy().squeeze())))
print("Predicted adversarial class index: " +str(pred_class[0]))
print("Adversarial confidence: {:.2f}".format(confidences[0]))


show_tensor_image(pert_image)

## 2. All Images

We are splitting the data according to the original confidences. For each of the intervals we craft adversarial examples at varying epsilon and report the confidence and accuracy of these attacks for the interval.


**TODO:**
- Generate adversaries from all samples if init prediction is correct. Track confidence
- Generate adversaries for sampels from confidence ranges and compare to other methods.

### 2.1 Confidences for all samples where initial prediction is correct

In [ ]:
result = pd.DataFrame()

# Find examples with correct initial prediction
samples = []

for sample in range(1000):
    image_clean, class_index = data_loader.dataset[sample]
    image_clean.unsqueeze_(0)
    class_index.unsqueeze_(0)
    
    predicted_classes, _, _ = predict(model, image_clean.cpu(), class_index, return_grad=False)
        
    if predicted_classes[0] == class_index.cpu().squeeze().numpy():
        samples.append(sample)
        clear_output(wait=True)
        print("Identified correct clean predictions: {}".format(len(samples)))

        
result["Sample"] = samples

conf1 = []
conf2 = []
conf3 = []
conf4 = []
conf5 = []
num_iter = []
class_indices_clean = []
class_indices_adver = []
adver_succ = []

# Attack only if initial prediction is correct
counter = 1
for sample in samples:
    clear_output(wait=True)
    print("Processing sample {} of {}".format(counter, len(samples)))
        
    # Get data
    image_clean, class_index = data_loader.dataset[sample]
    image_clean.unsqueeze_(0)
    class_index.unsqueeze_(0)

    # Change dimension from (1, 3, 224, 224) to (3, 224, 224):
    image_clean.squeeze_(0)

    # Deep Fool attack
    _, loop_i, class_index_clean, class_index_adver, pert_image = deepfool.deepfool(image_clean, googleNet)
    
    # Test image to get confidence
    pred_class, confidences, _ = predict(googleNet, pert_image, class_index)
    
    assert(class_index_adver == pred_class[0])
    
    # Check for adversarial success. 1 If class changed, 0 if not
    if pred_class[0] == class_index.cpu().squeeze(0).numpy():
        adver_succ.append(0)
    else:
        adver_succ.append(1)
        
    
    # Save stats for sample
    num_iter.append(loop_i)
    class_indices_clean.append(class_index_clean)
    class_indices_adver.append(pred_class[0])  
    conf1.append(confidences[0])
    conf2.append(confidences[1])
    conf3.append(confidences[2])
    conf4.append(confidences[3])
    conf5.append(confidences[4])
    
    counter += 1

    
# Save results
result["Clean class"] = class_indices_clean
result["Adversarial class"] = class_indices_adver
result["Success"] = adver_succ
result["Iterations"] = num_iter
result["Confidence 1"] = conf1
result["Confidence 2"] = conf2
result["Confidence 3"] = conf3
result["Confidence 4"] = conf4
result["Confidence 5"] = conf5
    
# Save results
#result.to_csv("results/DeepFool/DeepFool-adversarial_confidences_corr_only.csv")

print("Done")

In [ ]:
results_deepfool = pd.read_csv("results/DeepFool/DeepFool-adversarial_confidences_corr_only.csv", index_col=0)

### 2.2 Confidence ranges

## 3. Individual samples

-> Generate plots: Original image, adversary, top 5 confidence dist orig, top 5 conf dist adver

## References

[1] &emsp;  Moosavi-Dezfooli, S.-M., Fawzi, A., & Frossard, P. (2016). DeepFool: A Simple and Accurate Method to Fool Deep Neural Networks. 2016 IEEE Conference on Computer Vision and Pattern Recognition (CVPR).